In [1]:
import rasterio as rio
import numpy as np
from numba import njit
import pandas as pd
from tqdm import tqdm

In [2]:
combfile = '/media/eric/data/box/Projects/Auckland/5. Model/LSPC/ZonalRasterStatistics/HRU_Sliders/Raster_Layers_Combine/Tamaki/Combine.tif'
elefile = '/media/eric/data/box/Projects/Auckland/5. Model/LSPC/ZonalRasterStatistics/HRU_Sliders/Raster_Layers/Tamaki/elevation_tamaki.tif'
slopefile = '/media/eric/data/box/Projects/Auckland/5. Model/LSPC/ZonalRasterStatistics/HRU_Sliders/Raster_Layers/Tamaki/slope_tamaki.tif'
vegfile = '/media/eric/data/box/Projects/Auckland/5. Model/LSPC/ZonalRasterStatistics/HRU_Sliders/Raster_Layers/Tamaki/vegetation_tamaki.tif'

In [3]:
with rio.open(combfile) as src:
    combvals = src.read().squeeze()
    combnodata = src.nodatavals[0]
with rio.open(elefile) as src:
    elevals = src.read().squeeze()
    elenodata = src.nodatavals[0]
with rio.open(slopefile) as src:
    slopevals = src.read().squeeze()
    slopenodata = src.nodatavals[0]
with rio.open(vegfile) as src:
    vegvals = src.read().squeeze()
    vegnodata = src.nodatavals[0]

In [4]:
@njit
def avg_noncategoricals(combvals, elevals, slopevals, vegvals):
    
    unique_vals = np.unique(combvals)
    unique_vals = unique_vals[unique_vals!=combnodata]

    ele_avg = []
    slope_avg = []
    veg_avg = []
    
    rows, cols = combvals.shape
    
    for unique_val in unique_vals:
        ele_overlap = [np.float32(x) for x in range(0)]
        slope_overlap = [np.float32(x) for x in range(0)]
        veg_overlap = [np.float32(x) for x in range(0)]
        
        for row in range(0,rows):
            for col in range(0, cols):
                if combvals[row,col] == unique_val:
                    ele_overlap.append(elevals[row,col])
                    slope_overlap.append(slopevals[row,col])
                    veg_overlap.append(vegvals[row,col])
        
        ele_overlap = np.array(ele_overlap)
        slope_overlap = np.array(slope_overlap)
        veg_overlap = np.array(veg_overlap)
        
        ele_overlap = ele_overlap[ele_overlap != elenodata]
        slope_overlap = slope_overlap[slope_overlap != slopenodata]
        veg_overlap = veg_overlap[veg_overlap != vegnodata]
        
        ele_avg.append(np.mean(ele_overlap))
        slope_avg.append(np.mean(slope_overlap))
        veg_avg.append(np.mean(veg_overlap))
        
    return(ele_avg, slope_avg, veg_avg)


In [9]:
def avg_noncategoricals(combvals, elevals, slopevals, vegvals):
    
    unique_vals = np.unique(combvals)
    unique_vals = unique_vals[unique_vals!=combnodata]

    ele_avg = []
    slope_avg = []
    veg_avg = []
    
    
    for unique_val in tqdm(unique_vals):
        x,y = np.where(combvals==unique_val)
        
        ele_overlap = []
        slope_overlap = []
        veg_overlap = []
        
        ele_overlap = elevals[x,y]
        ele_overlap = ele_overlap[ele_overlap != elenodata]
        ele_avg.append(np.mean(ele_overlap))
        ele_overlap = None

        slope_overlap = slopevals[x,y]
        slope_overlap = slope_overlap[slope_overlap != slopenodata]
        slope_avg.append(np.mean(slope_overlap))
        slope_overlap = None

        veg_overlap = vegvals[x,y]
        veg_overlap = veg_overlap[veg_overlap != vegnodata]
        veg_avg.append(np.mean(veg_overlap))
        veg_overlap = None

        
    return(unique_vals, ele_avg, slope_avg, veg_avg)

In [10]:
unique_vals, ele_avg, slope_avg, veg_avg = avg_noncategoricals(combvals, elevals, slopevals, vegvals)

  6%|▌         | 501/9101 [02:28<39:58,  3.59it/s]  /home/eric/miniconda3/envs/distgeo/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/eric/miniconda3/envs/distgeo/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)
100%|██████████| 9101/9101 [41:40<00:00,  3.64it/s]


In [17]:
unique_vals = np.unique(combvals)
unique_vals = unique_vals[unique_vals!=combnodata]

In [21]:
StatsDF = pd.DataFrame({'Comb_ID':unique_vals, 'Elevation':ele_avg,'Slope':slope_avg,'Vegetation':veg_avg})

In [23]:
StatsDF.to_csv('/home/eric/projects/auckland/hru_creation/Tamaki_HRU_Table.csv',index=False)

In [24]:
import simpledbf

ModuleNotFoundError: No module named 'simpledbf'

In [26]:
import dbfread

ModuleNotFoundError: No module named 'dbfread'